**API`S**

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("df_transformado.csv")

In [16]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['id'],inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45375
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4485 non-null   object 
 1   budget                 45346 non-null  float64
 2   genres                 42962 non-null  object 
 3   id                     45346 non-null  int64  
 4   original_language      45335 non-null  object 
 5   overview               44405 non-null  object 
 6   popularity             45346 non-null  float64
 7   production_companies   33557 non-null  object 
 8   production_countries   39138 non-null  object 
 9   release_date           45346 non-null  object 
 10  revenue                45346 non-null  float64
 11  runtime                45100 non-null  float64
 12  spoken_languages       41580 non-null  object 
 13  status                 45266 non-null  object 
 14  tagline                20387 non-null  object 
 15  ti

API 1

In [9]:
def peliculas_mes(mes):

    months_translated= {
    'enero': 'January',
    'febrero': 'February',
    'marzo': 'March',
    'abril': 'April',
    'mayo': 'May',
    'junio': 'June',
    'julio': 'July',
    'agosto': 'August',
    'septiembre': 'September',
    'octubre': 'October',
    'noviembre': 'November',
    'diciembre': 'December'}  
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_mes = fechas[fechas.dt.strftime('%B').str.capitalize() == months_translated[str(mes).lower()]]
    respuesta = n_mes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}

In [10]:
peliculas_mes('marzo')

{'mes': 'marzo', 'cantidad': 3549}

API 2

In [11]:
def peliculas_dia(dia):
    
    day_translated= {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miercoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sabado': 'Saturday',
    'domingo': 'Sunday'}   
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_dia = fechas[fechas.dt.strftime('%A').str.capitalize() == day_translated[str(dia).lower()]]
    respuesta = n_dia.shape[0]
    return {'mes':dia, 'cantidad':respuesta}

In [12]:
peliculas_dia('lunes')

{'mes': 'lunes', 'cantidad': 3500}

API 3

In [13]:
def franquicia(franquicia):
    f_low=franquicia.lower()
    fran=df[['belongs_to_collection','revenue']].dropna(subset=['belongs_to_collection'])
    fran=fran[fran['belongs_to_collection'].map(str.lower).apply(lambda x: f_low in x)]
    cantidad=fran.shape[0]
    ganancia_total=fran['revenue'].sum()
    ganancia_promedio=fran['revenue'].mean()
    return {'franquicia':franquicia, 'cantidad':cantidad, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}

In [14]:
franquicia('toy story collection')

{'franquicia': 'toy story collection',
 'cantidad': 3,
 'ganancia_total': 1937890605.0,
 'ganancia_promedio': 645963535.0}

API 4

In [15]:
import unicodedata as unicodedata

In [25]:
def peliculas_pais(pais):
    m1 = df[['title', 'production_countries']]
    if isinstance(pais, str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        cantidad = m1['title'][m1['production_countries'].str.contains(pais)==True]
        cantidad = df['production_countries'].apply(lambda x: str(x).lower()).map(str.lower).apply(lambda x: pais in x)
        respuesta = cantidad.sum()
    return {'pais': pais, 'cantidad': respuesta}

In [26]:
peliculas_pais('colombia')

{'pais': 'colombia', 'cantidad': 34}

API 5

In [19]:
def productoras(productora):
    prod=df[['production_companies','revenue']].dropna()
    prod['production_companies']=prod['production_companies'].map(str.lower)
    prod=prod[prod.production_companies.str.contains(productora.lower(), regex=False)]
    cantidad=prod.shape[0]
    ganancia_total=prod['revenue'].sum()
    return {'productora':productora, 'ganancia_total':ganancia_total, 'cantidad':cantidad}

In [20]:
productoras('pixar')

{'productora': 'pixar', 'ganancia_total': 11188533734.0, 'cantidad': 52}

API 6

In [21]:
def retorno(pelicula):
    pelicula_df = df.loc[df['title'] == pelicula.title()]
    inversion = pelicula_df['budget'].iloc[0].item()
    ganancia = pelicula_df['revenue'].iloc[0].item()
    retorno= pelicula_df['return'].iloc[0].item()
    anio = pelicula_df['release_year'].iloc[0].item()
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio}

In [22]:
retorno('sabrina')

{'pelicula': 'sabrina',
 'inversion': 58000000.0,
 'ganancia': 0.0,
 'retorno': 0.0,
 'anio': 1995}

ML

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

Se limita el machine learning para que rendar lo pueda ejecutar sin problemas

In [27]:
ml=df.sample(n=2000, random_state=42)

In [28]:
tfidf= TfidfVectorizer(stop_words = 'english') 
ml['overview'] = ml['overview'].fillna('') 
tfidf_matrix = tfidf.fit_transform(ml['overview']) 
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
ml.drop_duplicates(inplace=True) 
ml = ml.reset_index(drop=True) 
indices = pd.Series(ml.index, index=ml['title'])

In [29]:
def recomendacion(titulo):
    idx = indices[titulo]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    recommendations=list(ml['title'].iloc[movie_indices].str.title())
    return {'lista recomendada': recommendations}


In [30]:
recomendacion ('Winning')

{'lista recomendada': ['The Visitors',
  'On A Clear Day',
  'Summer In The City',
  'Klown',
  'Gallipoli']}